**OLAP**: (Online analytical processing) any software that performs fast multidimensional analysis on large volumes of data.

**What is Apache Arrow:**
Apache arrow is a framework for defining in-memory columanar data that every processing engine can use.
- **processing engine** refers to a software system or framework that processes, analyzes, and transforms data. These engines are often used for tasks such as data querying, computation, and batch processing, and typically oeprate on large datasets, either in-memory or distributed systems across multiple machines.

    - Eg: Apache Spark, Pandas, Apache Flink

**Column database:**
What is a column databse? As the name suggests, a column (or columnar) database stores data organized into columns instead of rows on disk. A column database organize the data so that each column value is stored next to each other sequentially on a disk. 


The main advantage of a columnar database is that it can significantly reduce the amount of space required to store the data due to improvec compression ratios. In addition, columnar databases are much faster at processing analytic-type queries than traditional row-based databases. 
### How 
**Columnar Storage Format**
- the main technique that allows columnar databases and formats like Apache Arrow to be faster, especially for analytical queries can be attributed to their **columnar storage** and **vectorized processing**.

- **Data Locality:** In a columnar format, data is stored column by column rather than row by row. This means that when a query requests data from specific columns, only those columns need to be read from disk or memory, reducing I/O overhead and improving query performance

- **Efficient Data Access:** Columnar storage optimizes for read-heavy workloads typical in analytics. Since analytical queries often involve operations on a small subset of columns across many rows, the ability to access only relevant columns without reading the entire row structure is a key advantage.

**Vectorized  Processing:**
 - **SIMD (Single instruction, Multiple Data):** Columnar data is well-suited for vectorized processing, where the same operation is applied simultaneously to a block of data. This leverages the modern CPU architectures to process multiple data points in a single instruction 

- being able to use optimal compression algorithms for each data type because each column is the same type rather than a row of mixed data types. This not only reduces storage cost on disk but improves performance because fewer disk seeks are needed, and more data can fit into RAM.

- **Cache efficciency** The columnar format enhances cache efficiency because data in a single column is stored contiguously. This leads to better cache utilization, reducing the need for frequent memory fetches and speeding up data processing.


**Optimized Comphression:**
- **Type-specific Compression:** columnar databases and formats like Apache Arrow can apply compression algorithms tailored to the data type of each column, leading to better comphression ratios. This reduces storage footprint and can also improve performance by reducing the amount of data that needs to be read from disk or transferred over networks.

**In-Memory Processing (Apache Arrow Specific)**:

- Zero-Copy Data Sharing: Apache Arrow's format allows zero-copy reads and writes across different systems and languages, avoiding overhead of serialization and deserialization. This is particularly beneficial in data pipelines where data needs to be passed between multiple processing engines
- 



## Parqeut

Apache Parquet is a columnar storage format for big data procesing systems. Its designed for efficient storage and fast data access, and can handle complex data structures.

Parquet is an open source file format built to handle flat columnar storage data formats. Parquet operates well with complex data in large volumnes. It is known for its performant data compression and its ability to handle a wide varitey of encoding types. 

Parquet deploys Google's record-shredding and assembly algorithm that can address complex data strcutures within data storage. Some benefits:
- Fast queries that can fetch specific column values without reading the full data row
- Highly efficient column-wise compression
- High compartibility with OLAP

**Record-Shredding:** This technique breaks down complex, nested data structures (lists, maps, structs) into simpler, flat columns. Each element of the nested structure is stored in its own column, allowing the data to be stored inot a columnar format that is both space-efficient and fast to query. 
**Assembly Algorithm:** When querying the data, Parquet uses an assembly algorithm to reconstruct the orignal nested structure from these flattened columns. This process allows Parquet to efficiently store and retrieve complex data types while maintaining the benefits of columnar storage. 

### How is Parquet different from CSV?
- Parquet is column oriented and CSV is row oriented. Row-oriented formats are optimized for OLTP (online transactional processing workloads while column-oriented formats are better suited for analytical workloads
- 

### OLAP vs. OLTP
- OLAP system can process large amounts of data quickly
- OLTP systems ar desgined to handle large volumnes fo transactional data involving multiple users. Relational databases rapidly update, insert, or delete small amounts of data in real-time. Most OLTP systems are used for executing transactions such a banking transactions, hotel booking, etc..,

Many OLAP systems pull their data from OLTP databases via an ETL pipeline. Simply put organizations use OLTP systems to run the business while OLAP systems help them understand it. 




### What is Parquet?

Apache Parquet is an open source, column-oritented data file format for efficient data storage and retrieval. It excels in providing effective data compression and encoding schemes, which enhance performance and make it capable of handling complex, nested data structures at scale.  

**Use cases:* Often used in scenarios when large datasets need to be stored  efficienlty and queried frequently. It common in environments like data lakes, data warehouses, and big data pipelines 

**Technical:** 
- Parquet is built from the ground up with complex nested data strcutures in mind, and uses the record shredding and assembly algorithm described in the Dermel paper.
- Parquet is built to support very efficient compression and encoding schemes. 



**Compression** is the process of reducing the size of data to save storage space and redudce the time it takes to read or write data to disk. But what you're giving up is the CPU cycles. The trade off is between CPU(for compression/decompression)vs. IO  I/O operations (for reading/writing data)

- CPU vs. I/O Trade-off: Compress = reduce the data that needs to be transferred to and from disk (I/O operations); however compression and decompression data requires CPU resources. The decsion to use compression is a balance between reducing I/O overhead and th extra CPU cycles needed for the compression process.

- In an environment where I/O is the bottleneck (reading/wriing to/from disk), compression can improve overall performance despite the added CPU cost.

- In contrast, if CPU resources are limited or if real-time processing is cirtical, the added CPU overhead compression might outweigh the benefits



- since Parquet stores data in columnar format (i.e., column by column rather than row by row), each column contains data of the same type (integers, strings, dates). This homogeneity allows for more efficient compression.
- Parquet supports several compression algorithms such as Snappy, GZIP, and LZO. These algorithms take advantage of repeating patterns in the data within a column to compress it more effectively.


In [1]:
import os
import time
import pandas as pd
import numpy as np
import pyarrow as pa   # Import pyarrow for Parquet support
import pyarrow.parquet as pq

n_rows = 1_000_000
n_cols = 10

data = pd.DataFrame({
    f'col{i}': np.random.randint(0, 1000, size = n_rows)
    for i in range(n_cols)
})
data['category'] = np.random.choice(['A', 'B', 'C', 'D'], size = n_rows)


In [2]:
data

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,category
0,88,916,663,634,757,221,332,979,149,382,D
1,494,733,945,100,769,28,895,28,560,862,C
2,379,996,594,262,280,365,283,584,388,889,C
3,252,900,106,851,838,831,362,177,923,512,C
4,448,80,510,398,514,65,560,25,774,412,C
...,...,...,...,...,...,...,...,...,...,...,...
999995,197,331,899,638,872,418,46,49,685,833,A
999996,851,151,420,458,926,161,109,508,259,87,D
999997,561,958,297,617,274,399,292,942,545,405,B
999998,239,403,410,230,851,962,725,5,41,163,C


In [3]:
import time
import os


# CSV
csv_start_time = time.time()
data.to_csv('data.csv', index = False)
csv_end_time = time.time()

print(f"size: {os.path.getsize('data.csv') / (1024 * 1024):.2f} MB")
print(f"Time: {csv_end_time - csv_start_time:.2f} seconds")

size: 39.00 MB
Time: 1.64 seconds


## Parquet without Compression

In [8]:
parquet_start_time = time.time()
data.to_parquet('data.parquet', engine= 'pyarrow', compression = None, index = False)
parquet_end_time = time.time()

print(f"size: {os.path.getsize('data.parquet') / (1024 * 1024):.2f} MB")
print(f"Time: {parquet_end_time - parquet_start_time:.2f} seconds")

size: 12.26 MB
Time: 0.22 seconds


In [9]:
# Write the DataFrame to a Parquet file with compression
parquet_compressed_start_time = time.time()
data.to_parquet('data_compressed.parquet', engine='pyarrow', compression='snappy', index=False)
parquet_compressed_end_time = time.time()

print(f"Parquet file size (with compression): {os.path.getsize('data_compressed.parquet') / (1024 * 1024):.2f} MB")
print(f"Time taken to write Parquet (with compression): {parquet_compressed_end_time - parquet_compressed_start_time:.2f} seconds")


Parquet file size (with compression): 12.23 MB
Time taken to write Parquet (with compression): 0.22 seconds


In [11]:
import pandas as pd
import numpy as np

# Set the number of rows
n_rows = 500_000  # Half a million entries to simulate a large census-like dataset

# Generate demographic data
demographic_data = {
    'age': np.random.randint(18, 90, size=n_rows),  # Ages between 18 and 90
    'gender': np.random.choice(['Male', 'Female', 'Other'], size=n_rows),  # Gender categories
    'marital_status': np.random.choice(['Single', 'Married', 'Divorced', 'Widowed'], size=n_rows)
}

# Generate income data
income_data = {
    'household_income': np.random.normal(70000, 25000, size=n_rows),  # Normally distributed income
    'individual_income': np.random.normal(40000, 15000, size=n_rows),
    'income_category': pd.cut(np.random.normal(70000, 25000, size=n_rows),
                              bins=[0, 30000, 60000, 100000, 150000, np.inf],
                              labels=['Low', 'Lower-Middle', 'Middle', 'Upper-Middle', 'High'])
}

# Generate geographical data
provinces = ['Ontario', 'Quebec', 'British Columbia', 'Alberta', 'Manitoba', 
             'Saskatchewan', 'Nova Scotia', 'New Brunswick', 'Newfoundland and Labrador', 
             'Prince Edward Island', 'Northwest Territories', 'Yukon', 'Nunavut']
geographical_data = {
    'province': np.random.choice(provinces, size=n_rows),
    'city': np.random.choice(['Toronto', 'Montreal', 'Vancouver', 'Calgary', 'Edmonton', 
                              'Ottawa', 'Quebec City', 'Winnipeg', 'Halifax'], size=n_rows),
    'postal_code': [''.join([np.random.choice(list('ABCDEFGHIJKLMNOPQRSTUVWXYZ')),
                             np.random.choice(list('0123456789')),
                             np.random.choice(list('ABCDEFGHIJKLMNOPQRSTUVWXYZ')),
                             ' ',
                             np.random.choice(list('0123456789')),
                             np.random.choice(list('ABCDEFGHIJKLMNOPQRSTUVWXYZ')),
                             np.random.choice(list('0123456789'))])
                    for _ in range(n_rows)]
}

# Generate employment data
employment_data = {
    'employment_status': np.random.choice(['Employed', 'Unemployed', 'Not in Labour Force'], size=n_rows),
    'occupation': np.random.choice(['Management', 'Business', 'Science', 'Health', 
                                    'Education', 'Art', 'Sales', 'Trades', 'Manufacturing'], size=n_rows),
    'industry': np.random.choice(['Agriculture', 'Manufacturing', 'Retail', 'Construction', 
                                  'Finance', 'Education', 'Healthcare', 'Tech', 'Government'], size=n_rows)
}

# Generate education data
education_data = {
    'education_level': np.random.choice(['No Diploma', 'High School', 'College', 'Bachelor', 'Master', 'Doctorate'], size=n_rows)
}

# Generate census data
census_data = {
    'household_size': np.random.randint(1, 6, size=n_rows),  # Household size between 1 and 5
    'dwelling_type': np.random.choice(['House', 'Apartment', 'Townhouse', 'Condo', 'Duplex'], size=n_rows)
}

# Combine all data into a DataFrame
data = pd.DataFrame({**demographic_data, **income_data, **geographical_data, 
                     **employment_data, **education_data, **census_data})


   age  gender marital_status  household_income  individual_income  \
0   60   Other         Single      48742.636095       68943.117809   
1   37    Male         Single      78697.969919       26020.048371   
2   65    Male        Widowed      33917.136328       87610.882576   
3   83  Female       Divorced      71738.193542       44953.755703   
4   23    Male       Divorced      49044.507036       39434.799359   

  income_category      province         city postal_code    employment_status  \
0          Middle       Alberta      Toronto     S1W 2G4           Unemployed   
1          Middle  Saskatchewan     Winnipeg     L2F 4Z6  Not in Labour Force   
2    Lower-Middle         Yukon      Calgary     K4K 9Y1             Employed   
3          Middle         Yukon  Quebec City     O6H 7Y4           Unemployed   
4    Lower-Middle      Manitoba     Edmonton     T8D 7M3           Unemployed   

      occupation      industry education_level  household_size dwelling_type  
0     Managem

In [12]:
data

,age,gender,marital_status,household_income,individual_income,income_category,province,city,postal_code,employment_status,occupation,industry,education_level,household_size,dwelling_type
0,60,Other,Single,48742.636095,68943.117809,Middle,Alberta,Toronto,S1W 2G4,Unemployed,Management,Agriculture,Bachelor,3,Condo
1,37,Male,Single,78697.969919,26020.048371,Middle,Saskatchewan,Winnipeg,L2F 4Z6,Not in Labour Force,Business,Agriculture,Doctorate,2,Duplex
2,65,Male,Widowed,33917.136328,87610.882576,Lower-Middle,Yukon,Calgary,K4K 9Y1,Employed,Education,Tech,Bachelor,2,Townhouse
3,83,Female,Divorced,71738.193542,44953.755703,Middle,Yukon,Quebec City,O6H 7Y4,Unemployed,Education,Construction,Master,1,House
4,23,Male,Divorced,49044.507036,39434.799359,Lower-Middle,Manitoba,Edmonton,T8D 7M3,Unemployed,Manufacturing,Tech,No Diploma,4,Condo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,75,Female,Divorced,34158.445942,32957.425234,Lower-Middle,Saskatchewan,Montreal,C4O 5Y1,Unemployed,Business,Tech,Doctorate,1,Duplex
499996,77,Female,Widowed,65678.480391,46158.309234,Middle,Saskatchewan,Halifax,P2N 6G2,Employed,Management,Manufacturing,Master,5,Townhouse
499997,60,Other,Single,67554.800350,29964.153289,Middle,Manitoba,Toronto,K0P 6Q8,Employed,Manufacturing,Manufacturing,Master,4,Condo
499998,81,Other,Single,71621.728631,18202.168957,Middle,Nunavut,Montreal,Z7O 5X8,Unemployed,Management,Manufacturing,No Diploma,4,Townhouse


In [13]:
import os
import time

# Save as CSV
csv_start_time = time.time()
data.to_csv('statistics_canada_data.csv', index=False)
csv_end_time = time.time()

print(f"CSV file size: {os.path.getsize('statistics_canada_data.csv') / (1024 * 1024):.2f} MB")
print(f"Time taken to write CSV: {csv_end_time - csv_start_time:.2f} seconds")

# Save as Parquet without Compression
parquet_start_time = time.time()
data.to_parquet('statistics_canada_data.parquet', engine='pyarrow', compression=None, index=False)
parquet_end_time = time.time()

print(f"Parquet file size (no compression): {os.path.getsize('statistics_canada_data.parquet') / (1024 * 1024):.2f} MB")
print(f"Time taken to write Parquet (no compression): {parquet_end_time - parquet_start_time:.2f} seconds")

# Save as Parquet with Snappy Compression
snappy_start_time = time.time()
data.to_parquet('statistics_canada_data_snappy.parquet', engine='pyarrow', compression='snappy', index=False)
snappy_end_time = time.time()

print(f"Parquet file size (with Snappy): {os.path.getsize('statistics_canada_data_snappy.parquet') / (1024 * 1024):.2f} MB")
print(f"Time taken to write Parquet (with Snappy): {snappy_end_time - snappy_start_time:.2f} seconds")

# Save as Parquet with GZIP Compression
gzip_start_time = time.time()
data.to_parquet('statistics_canada_data_gzip.parquet', engine='pyarrow', compression='gzip', index=False)
gzip_end_time = time.time()

print(f"Parquet file size (with GZIP): {os.path.getsize('statistics_canada_data_gzip.parquet') / (1024 * 1024):.2f} MB")
print(f"Time taken to write Parquet (with GZIP): {gzip_end_time - gzip_start_time:.2f} seconds")


CSV file size: 68.63 MB
Time taken to write CSV: 1.73 seconds
Parquet file size (no compression): 16.07 MB
Time taken to write Parquet (no compression): 0.34 seconds
Parquet file size (with Snappy): 14.43 MB
Time taken to write Parquet (with Snappy): 0.35 seconds
Parquet file size (with GZIP): 12.29 MB
Time taken to write Parquet (with GZIP): 1.00 seconds
